In [146]:
import os
import re
import json 

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
from matplotlib import colors
import seaborn as sns 
import plotly.graph_objects as go 
import plotly.express as px 
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots

from shapely.geometry import Polygon 
#import geopandas as gpd 
import folium
import folium.plugins

from IPython.display import HTML, display

init_notebook_mode(connected=True)

In [41]:
COUNTRY = 'India'

In [8]:
cities_dir = 'Cities/Cities Responses/'
cc_dir = 'Corporations/Corporations Responses/Climate Change/'
ws_dir = 'Corporations/Corporations Responses/Water Security/'

c_18 = pd.read_csv(cities_dir + '2018_Full_Cities_Dataset.csv')
c_19 = pd.read_csv(cities_dir + '2019_Full_Cities_Dataset.csv')
c_20 = pd.read_csv(cities_dir + '2020_Full_Cities_Dataset.csv')

cc_18 = pd.read_csv(cc_dir + '2018_Full_Climate_Change_Dataset.csv')
cc_19 = pd.read_csv(cc_dir + '2019_Full_Climate_Change_Dataset.csv')
cc_20 = pd.read_csv(cc_dir + '2020_Full_Climate_Change_Dataset.csv')

ws_18 = pd.read_csv(ws_dir + '2018_Full_Water_Security_Dataset.csv')
ws_19 = pd.read_csv(ws_dir + '2019_Full_Water_Security_Dataset.csv')
ws_20 = pd.read_csv(ws_dir + '2020_Full_Water_Security_Dataset.csv')

In [15]:
cities_dir_disc = 'Cities/Cities Disclosing/'
cc_dir_disc = 'Corporations/Corporations Disclosing/Climate Change/'
ws_dir_disc = 'Corporations/Corporations Disclosing/Water Security/'

c_18_disc = pd.read_csv(cities_dir_disc + '2018_Cities_Disclosing_to_CDP.csv')
c_19_disc = pd.read_csv(cities_dir_disc + '2019_Cities_Disclosing_to_CDP.csv')
c_20_disc = pd.read_csv(cities_dir_disc + '2020_Cities_Disclosing_to_CDP.csv')

cc_18_disc = pd.read_csv(cc_dir_disc + '2018_Corporates_Disclosing_to_CDP_Climate_Change.csv')
cc_19_disc = pd.read_csv(cc_dir_disc + '2019_Corporates_Disclosing_to_CDP_Climate_Change.csv')
cc_20_disc = pd.read_csv(cc_dir_disc + '2020_Corporates_Disclosing_to_CDP_Climate_Change.csv')

ws_18_disc = pd.read_csv(ws_dir_disc + '2018_Corporates_Disclosing_to_CDP_Water_Security.csv')
ws_19_disc = pd.read_csv(ws_dir_disc + '2019_Corporates_Disclosing_to_CDP_Water_Security.csv')
ws_20_disc = pd.read_csv(ws_dir_disc + '2020_Corporates_Disclosing_to_CDP_Water_Security.csv')

In [181]:
def clean_city_name(city):
    return re.sub(r'City|of|Municipal|Corporation|Metropolitan|Area', '', city).strip()

In [180]:
india_18 = c_18[c_18['Country'] == COUNTRY]
india_19 = c_19[c_19['Country'] == COUNTRY]
india_20 = c_20[c_20['Country'] == COUNTRY]

print(india_20['Parent Section'].value_counts())
print(india_20['Section'].value_counts())
india.head()

City-wide Emissions                  724
Emissions Reduction                  424
Climate Hazards and Vulnerability    171
Opportunities                        140
Adaptation                           115
Water Security                        89
Local Government Emissions            62
Governance and Data Management        50
Introduction                          18
Name: Parent Section, dtype: int64
City-wide GHG Emissions Data                      625
Transport                                         371
Mitigation Actions                                245
Mitigation Target setting                         145
Climate Hazards                                   129
Energy                                            106
Food                                               82
Finance and Economic Opportunities                 79
Buildings                                          78
Waste                                              62
Local Government Operations GHG Emissions Data     50
Wa

,Questionnaire,Year Reported to CDP,Account Number,Organization,Country,CDP Region,Parent Section,Section,Question Number,Question Name,Column Number,Column Name,Row Number,Row Name,Response Answer,Comments,File Name,Last update
392,Cities 2020,2020,31157,City of Delhi,India,South and West Asia,NaN,Transport,10.14,Please provide city-wide average air pollution...,4,Average concentration for third most recent ye...,6,O3 (Daily maximum 8 hour mean),NaN,NaN,NaN,09/07/2020 09:45:36 AM
789,Cities 2020,2020,35905,Corporation of Chennai,India,South and West Asia,Emissions Reduction,Mitigation Target setting,5.0c,Please provide details of your total city-wide...,12,Percentage of target achieved,1,NaN,Question not applicable,NaN,NaN,09/07/2020 09:45:36 AM
1011,Cities 2020,2020,35905,Corporation of Chennai,India,South and West Asia,NaN,Transport,10.8,Do you have a loading / unloading Restricted Z...,1,Response,1,Please complete,Question not applicable,NaN,NaN,09/07/2020 09:45:36 AM
2846,Cities 2020,2020,35905,Corporation of Chennai,India,South and West Asia,Emissions Reduction,Mitigation Target setting,5.0d,Please provide details of your total city-wide...,16,Target meets initial GCoM validation criteria,0,NaN,Question not applicable,NaN,NaN,09/07/2020 09:45:36 AM
2865,Cities 2020,2020,31157,City of Delhi,India,South and West Asia,Emissions Reduction,Mitigation Actions,5.4,Describe the anticipated outcomes of the most ...,6,Energy savings (MWh),3,NaN,NaN,NaN,NaN,09/07/2020 09:45:36 AM


# CITY-WIDE EMISSIONS 2020
## Delhi & Chennai
### 724 question responses

In [207]:
NUM_Q = 'Question Number'
NUM_R = 'Row Number'
NUM_C = 'Column Number'

NAME_Q = 'Question Name'
NAME_R = 'Row Name'
NAME_C = 'Column Name'

ANS = 'Response Answer'

In [225]:
city_emissions = india_20[india_20['Parent Section'] == 'City-wide Emissions']
delhi = city_emissions[city_emissions['Organization'] == 'City of Delhi']
chennai = city_emissions[city_emissions['Organization'] == 'Corporation of Chennai']

print('Chennai:', len(chennai.index), 'questions answered')
print('Delhi:', len(delhi.index), 'questions answered')
print('---------------------------------------------------------------------')

print('Q(4.0) Does your city have a city-wide emissions inventory to report?')
print('Options: [Yes, In Progress, Intending to Undertake (2 years), Not Intending to Undertake]\n')
#print('Chennai:', chennai[chennai[NUM_Q] == '4.0'].at[df,ANS])
#print('Delhi:', delhi[delhi[NUM_Q] == '4.0'].at[0,ANS])
print('---------------------------------------------------------------------')

print('Q(4.1) Please state the dates of the accounting year or 12-month period for which you are reporting your latest city-wide GHG emissions inventory.\n')
print('Chennai:', 'FROM %s TO %s' % (*chennai[chennai[NUM_Q] == '4.1'][ANS].to_list(),))
print('Delhi:', 'FROM %s TO %s' % (*delhi[delhi[NUM_Q] == '4.1'][ANS].to_list(),))
print('---------------------------------------------------------------------')

print('Q(4.2) Please indicate the category that best describes the boundary of your city-wide GHG emissions inventory.\n')

def labelize(row_or_col):
    if row_or_col == 'row':
        return lambda row: (row[NUM_R], f'{row[NUM_R]}. {row[NAME_R]}')
    return lambda col: (col[NUM_C], f'{col[NUM_C]}. {col[NAME_C]}')

chennai_4_2 = chennai[chennai[NUM_Q] == '4.2']
for row, name_r in chennai_4_2.apply(labelize('row'), axis=1).unique():
    print(name_r)
    row_df = chennai_4_2[chennai_4_2[NUM_R] == row]
    for col, name_c in row_df.apply(labelize('col'), axis=1).unique():
        print('\t', name_c)
        col_df = row_df[row_df[NUM_C] == col]
        idx = col_df.index[0]
        print('\t\tAnswer:', col_df.at[idx,ANS])
        if col_df.at[idx,'Comments'] is not np.NaN:
            print('\t\tComments:', col_df.at[idx,'Comments'])
        if col_df.at[idx,'File Name'] is not np.NaN:
            print('\t\tFurther info @:', col_df.at[idx,'File Name'])
delhi_4_2 = delhi[delhi[NUM_Q] == '4.2']
#for row in chennai_4_2[NUM_R].unique()
print('Chennai:\n', f'1. {chennai_4_2[chennai_4_2[NUM_C] == 1][NAME_Q]}:')
print('---------------------------------------------------------------------')

chennai_4_2

Chennai: 356 questions answered
Delhi: 368 questions answered
---------------------------------------------------------------------
Q(4.0) Does your city have a city-wide emissions inventory to report?
Options: [Yes, In Progress, Intending to Undertake (2 years), Not Intending to Undertake]

---------------------------------------------------------------------
Q(4.1) Please state the dates of the accounting year or 12-month period for which you are reporting your latest city-wide GHG emissions inventory.

Chennai: FROM 2018-04-01 TO 2019-03-31
Delhi: FROM 2014-01-01 TO 2015-01-01
---------------------------------------------------------------------
Q(4.2) Please indicate the category that best describes the boundary of your city-wide GHG emissions inventory.

1. Please explain
	 1. Boundary of inventory relative to city boundary (reported in 0.1)
		Answer: Same – covers entire city and nothing else
	 2. Excluded sources / areas
		Answer: Does not include Scope 3 emissions.
	 3. Explana

,Questionnaire,Year Reported to CDP,Account Number,Organization,Country,CDP Region,Parent Section,Section,Question Number,Question Name,Column Number,Column Name,Row Number,Row Name,Response Answer,Comments,File Name,Last update
101257,Cities 2020,2020,35905,Corporation of Chennai,India,South and West Asia,City-wide Emissions,City-wide GHG Emissions Data,4.2,Please indicate the category that best describ...,1,Boundary of inventory relative to city boundar...,1,Please explain,Same – covers entire city and nothing else,NaN,NaN,09/07/2020 09:45:36 AM
174607,Cities 2020,2020,35905,Corporation of Chennai,India,South and West Asia,City-wide Emissions,City-wide GHG Emissions Data,4.2,Please indicate the category that best describ...,2,Excluded sources / areas,1,Please explain,Does not include Scope 3 emissions.,NaN,NaN,09/07/2020 09:45:36 AM
573543,Cities 2020,2020,35905,Corporation of Chennai,India,South and West Asia,City-wide Emissions,City-wide GHG Emissions Data,4.2,Please indicate the category that best describ...,3,Explanation of boundary choice where the inven...,1,Please explain,The Inventory Boundary and the City Boundary a...,NaN,NaN,09/07/2020 09:45:36 AM


101257    1. Please explain
174607    1. Please explain
573543    1. Please explain
dtype: object

In [86]:
india_18_orgs = india_18['Organization'].value_counts()
india_19_orgs = india_19['Organization'].value_counts()
india_20_orgs = india_20['Organization'].value_counts()

fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=('2018', '2019', '2020'), x_title='Cities', y_title='# of responses')
fig.add_trace(go.Bar(x=india_18_orgs.index.map(clean_city_name), y=india_18_orgs), row=1, col=1)
fig.add_trace(go.Bar(x=india_19_orgs.index.map(clean_city_name), y=india_19_orgs), row=1, col=2)
fig.add_trace(go.Bar(x=india_20_orgs.index.map(clean_city_name), y=india_20_orgs), row=1, col=3)
fig.update_layout(
    title={
        'text':'Indian Cities Disclosing to CDP', 
        'font_size':24, 'yanchor':'top', 'x':0.5},
    showlegend=False,
    xaxis={'tickangle':-70},
    xaxis2={'domain':[0.35,0.8]},
    xaxis3={'domain':[0.85,1]})
fig.layout.annotations[1]['xshift'] = 30
fig.layout.annotations[2]['xshift'] = 40
fig.layout.annotations[3]['yshift'] = -50
fig.layout.annotations[3]['xshift'] = 30
fig.show()

In [177]:
from jinja2 import Template
from folium.features import DivIcon

maps_responses = [folium.Map(
    width='33.3%' if left != '33.3%' else '33.4%',
    left=left, position='absolute',
    zoom_start=4, zoom_control=False, max_zoom=7,
    tiles='CartoDB dark_matter',
    location=[23,80]
) for left in ('0%', '33.3%', '66.7%')]

with open('constants.json') as f:
    latlong = json.load(f)

cmap = plt.get_cmap('spring_r')
norm = colors.Normalize(vmin=295, vmax=1517)
for i, in_orgs in enumerate((india_18_orgs, india_19_orgs, india_20_orgs)):
    for city in in_orgs.index:
        count = in_orgs[city]
        color = colors.to_hex(cmap(norm(count)))
        folium.CircleMarker(
            location=latlong[clean_city_name(city)],
            radius=30 * count / 1517,
            popup=f'{city}<br/>({count})',
            color=color,
            fill=True,
            fill_color=color
        ).add_to(maps_responses[i])
for i, year in enumerate((2018, 2019, 2020)):
    folium.map.Marker(
        [30, 80],
        icon=DivIcon(
            icon_size=(155,50), 
            icon_anchor=(-40, 100), 
            html=f'<div style="font-size:24px;"><p style="color:white;">{year}</p></div>')
    ).add_to(maps_responses[i])


# Interactive Map with separately pannable sections for each year
from branca.element import Figure
fig = Figure()
for map_ in maps_responses:
    fig.add_child(map_)
#display(HTML(fig._repr_html_()))

class TriMap(folium.plugins.DualMap):
    _template = Template("""
        {% macro script(this, kwargs) %}
            {{ this.m1.get_name() }}.sync({{ this.m2.get_name() }});
            {{ this.m2.get_name() }}.sync({{ this.m1.get_name() }});
            {{ this.m3.get_name() }}.sync({{ this.m1.get_name() }});
            {{ this.m1.get_name() }}.sync({{ this.m3.get_name() }});
            {{ this.m3.get_name() }}.sync({{ this.m2.get_name() }});
            {{ this.m2.get_name() }}.sync({{ this.m3.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, *maps):
        super(folium.plugins.DualMap, self).__init__()
        self.m1 = maps[0]
        self.m2 = maps[1]
        self.m3 = maps[2]
        figure = Figure()
        figure.add_child(self.m1)
        figure.add_child(self.m2)
        figure.add_child(self.m3)
        figure.add_child(self)

        self.children_for_m2 = []
        self.children_for_m2_copied = []

TriMap(*maps_responses)